In [ ]:
%matplotlib inline


Usage of the SurrogateTool for semi-analytical beam Model
=========================================================

:class:`~.SurrogateTool` provides methods for the generation of surrogate models.
It identifies the best surrogate and associated quality measures within a list of
default surrogate candidates.
It uses either a default list of candidates or candidates specified by the user.
The :class:`~.BendingTestAnalytical` is used to illustrate the use of this tool.


Import the packages
--------------------------



In [ ]:
from __future__ import annotations

import logging
import pprint

from gemseo.post.dataset.scatter_plot_matrix import ScatterMatrix
from numpy import array

from vimseo import EXAMPLE_RUNS_DIR_NAME
from vimseo.api import activate_logger
from vimseo.api import create_model
from vimseo.core.model_settings import IntegratedModelSettings
from vimseo.tools.doe.doe import DOETool
from vimseo.tools.space.space_tool import SpaceTool
from vimseo.tools.surrogate.surrogate import SurrogateTool

activate_logger(level=logging.INFO)

Model and parameter space
--------------------------
Load the model (here a simple analytical beam model):



In [ ]:
model_name = "BendingTestAnalytical"
load_case = "Cantilever"
model = create_model(
    model_name,
    load_case,
    model_options=IntegratedModelSettings(
        directory_archive_root=f"../../../{EXAMPLE_RUNS_DIR_NAME}/archive/surrogate",
        directory_scratch_root=f"../../../{EXAMPLE_RUNS_DIR_NAME}/scratch/surrogate",
        cache_file_path=f"../../../{EXAMPLE_RUNS_DIR_NAME}/caches/surrogate/{model_name}_{load_case}_cache.hdf",
    ),
)

The SpaceTool provides methods to build parameter spaces.
One can check available builders first:



In [ ]:
space_tool = SpaceTool(working_directory="SpaceTool_results")

Build a parameter space based on custom minimum and maximum values.
This parameter space has 2 scalar variables with large variations.



In [ ]:
space_tool.execute(
    distribution_name="OTTriangularDistribution",
    space_builder_name="FromMinAndMax",
    minimum_values={
        # "length": 200.0,
        # "width": 5.0,
        "height": 5.0,
        "imposed_dplt": 0.0,
        # "relative_dplt_location": 0.5,
    },
    maximum_values={
        # "length": 1000.0,
        # "width": 50.0,
        "height": 50.0,
        "imposed_dplt": 20.0,
        # "relative_dplt_location": 1.0,
    },
)
print(space_tool.parameter_space)

Create a :class:`~.gemseo.datasets.dataset.Dataset` (from a DOE)
----------------------------------------------------------------
The :class:`~.SurrogateTool` used a dataset from which the surrogate model is generated.
In this example we build the dataset by
creating a DOE. However, the dataset could be manually created,
or imported from a HDF5 cache file.
To generate a dataset from a DOE we use the :class:`~.DOETool`, along with the :class:`~.SpaceTool`.



In [ ]:
doe_tool = DOETool(working_directory="doe_tool_results")
dataset = doe_tool.execute(
    model=model,
    parameter_space=space_tool.parameter_space,
    output_names=["reaction_forces"],
    algo="OT_OPT_LHS",
    n_samples=200,
).dataset
print(dataset)

The generated dataset can also be plotted in a scatter matrix



In [ ]:
scatter_matrix = ScatterMatrix(dataset)
scatter_matrix.execute(save=False, show=True)
fig = scatter_matrix.figures[0]

fig

Building a surrogate model
--------------------------



In [ ]:
surrogate_tool = SurrogateTool(working_directory="surrogate_tool_results")

Settings of the tool can be checked (other options can be redefined throughout
:code:`surrogate_tool.update_options()`):



In [ ]:
pprint.pprint(surrogate_tool.options)

There are two main ways for using the SurrogateTool:

* One wants to use a specific surrogate function (e.g. 2nd order polynomial regression)

.. code::

   surrogates = surrogate_tool.execute(model=model, doe_results=dataset,
                                       algo="PolynomialRegressor", algo_options={"degree": 2, "fit_intercept": True}
                                       output_names=["dplt_at_force_location"])



* One wants to select the best surrogate function over a set of candidates and their
parameters (default candidates and other options can be redefined throughout
:code:`surrogate_tool.update_options()`). A quality measure (associated with an
evaluation method) is used. By default, it uses "Mean Squared Error" with
"kfolds" evaluation method.



In [ ]:
surrogate_results = surrogate_tool.execute(
    model=model,
    dataset=dataset,
    output_names=["reaction_forces"],
    quality_measures=["MSEMeasure", "R2Measure"],
)
surrogate_tool.save_results()
surrogate = surrogate_results.model
print(surrogate_tool.selected_algo)

.. note::
   It is possible to redefine candidate algorithms and their options (see |gemseo|
   documentation for available algorithms).

   .. code::

       custom_candidates = {"RBFRegressor": {},
                            "LinearRegressor", {'fit_intercept': [True, False]},
                            "PolynomialRegressor", {'degree': [4, 5], 'fit_intercept': [True, False]}
                            }
       surrogate_tool.set_candidates(custom_candidates)




The surrogate model can then be used in place of the model (outputs restricted to "dplt_at_force_location"):



In [ ]:
input_data = {
    # "young_modulus": array([150000.0]),
    # "length": array([5450.0]),
    # "width": array([245.0]),
    "height": array([35.0]),
    "imposed_dplt": array([10.0]),
}
output = surrogate.execute(input_data)
output["reaction_forces"]

which can be compared to prediction of the original model:



In [ ]:
true_output = model.execute(input_data)
true_output["reaction_forces"]

Analysing the quality of surrogates
-----------------------------------
SurrogateTool comes with functions to analyse several quality measures that could be complementary.
One can define the quality measures to be evaluated, as well as the evaluation methods
(alternatively, use default selection). Here, default ones are considered.

Qualities are computed at each call to :meth:`~.execute()` method, but can be computed
again with other parameters by :meth:`~.compute_quality()` method.

Then, show the quality measures:



In [ ]:
print(surrogate_tool.result)

Predictions versus observations can be plotted:



In [ ]:
figures = surrogate_tool.plot_results(
    surrogate_tool.result,
    output_names=["reaction_forces"],
    show=True,
    save=False,
)

The user is invited to visit the |gemseo| documentation for a complete view of the
available functionalities for surrogate generation and exploitation.

